In [ ]:
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
from itertools import combinations
from math import factorial

from skopt import BayesSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import shap
from alibi.explainers import KernelShap


import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
def get_hyperparameter_scores( search ):
    results_df = pd.DataFrame(search.cv_results_)
    results_df = results_df.sort_values(by=["rank_test_score"])
    results_df = results_df.set_index(
        results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
    ).rename_axis("kernel")
    
    return results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

In [ ]:
iris = load_iris()
X, y = load_iris( return_X_y = True )

scaler   = StandardScaler().fit( X )
X_normed = scaler.transform( X )
Xtrain, Xtest, ytrain, ytest = train_test_split( X_normed, y, train_size = 0.75, test_size = .25, random_state = 0 )

cv = RepeatedStratifiedKFold( n_splits = 10, n_repeats = 10, random_state = 0 ) # A 10x 10-fold cross-validation

In [ ]:
svc_as = BayesSearchCV(
                        SVC(),
                        {
                            'C': (1e-6, 1e+6, 'log-uniform'),
                            'gamma': (1e-6, 1e+1, 'log-uniform'),
                            'degree': (1, 8),  # integer valued parameter
                            'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
                        },
                        n_iter=10,
                        cv = cv
                     )

svc_as.fit( Xtrain, ytrain )

print("params    : %s" % svc_as.best_params_)
print("val. score: %s" % svc_as.best_score_)
print("test score: %s" % svc_as.score(Xtest, ytest))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

random_state = check_random_state( 0 )
permutation  = random_state.permutation( X.shape[ 0 ] )

X = X[ permutation ]
y = y[ permutation ]
X = X.reshape( ( X.shape[ 0 ], -1 ) )

Xtrain, Xtest, ytrain, ytest = train_test_split( X, y, train_size = train_samples, test_size = test_samples )

scaler = StandardScaler()
Xtrain = scaler.fit_transform( Xtrain )
Xtest  = scaler.transform( Xtest )

In [ ]:
svc_rs = RandomizedSearchCV( 
                            SVC(), 
                            {
                              'C'     : stats.loguniform(1e-6, 1e+6),
                              'gamma' : stats.loguniform(1e-6, 1e+1),
                              'degree': stats.randint(1, 8),         
                              'kernel': ['linear', 'poly', 'rbf'], 
                            }, 
                            n_iter = 10, 
                            cv = cv, 
                            n_jobs = 6 )
svc_rs.fit( Xtrain, ytrain )

print("params    : %s" % svc_rs.best_params_)
print("val. score: %s" % svc_rs.best_score_)
print("test score: %s" % svc_rs.score(Xtest, ytest))

In [ ]:
svc_gs = GridSearchCV( 
                       SVC(), 
                       {
                         'C'     : np.power(10, np.arange(-4, 1, dtype=float)),
                         'gamma' : np.power(10, np.arange(-4, 1, dtype=float)),
                         'degree': np.arange(1, 9, dtype = int ),         
                         'kernel': ['linear', 'poly', 'rbf'], 
                       }, 
                       cv = cv, 
                       n_jobs = 6 )

svc_gs.fit( Xtrain, ytrain )

print("params    : %s" % svc_gs.best_params_)
print("val. score: %s" % svc_gs.best_score_)
print("test score: %s" % svc_gs.score(Xtest, ytest))

In [ ]:
lgr_as = BayesSearchCV(
                        LogisticRegression(),
                        {
                            'C': (1e-6, 1e+6, 'log-uniform'),
                            'tol': (1e-6, 1e+1, 'log-uniform'),
                            'max_iter': (100, 400),  
                            'solver': ['sag', 'saga', 'newton-cg'],
                        },
                        n_iter=10,
                        cv = cv
                     )

lgr_as.fit( Xtrain, ytrain )

print("params    : %s" % lgr_as.best_params_)
print("val. score: %s" % lgr_as.best_score_)
print("test score: %s" % lgr_as.score(Xtest, ytest))

In [ ]:
lgr_rs = RandomizedSearchCV(
                        LogisticRegression(),
                        {
                            'C':   stats.loguniform(1e-6, 1e+6),
                            'tol': stats.loguniform(1e-6, 1e+6),
                            'max_iter': stats.randint(100, 400),  # integer valued parameter
                            'solver': ['sag', 'saga', 'newton-cg'],  # categorical parameter
                        },
                        n_iter=10,
                        cv = cv
                     )

lgr_rs.fit(Xtrain, ytrain)

print("params    : %s" % lgr_rs.best_params_)
print("val. score: %s" % lgr_rs.best_score_)
print("test score: %s" % lgr_rs.score(Xtest, ytest))

In [ ]:
lgr_gs = GridSearchCV(
                        LogisticRegression(),
                        {
                            'C'       : np.power(10, np.arange(-4, 1, dtype=float)),
                            'tol'     : np.power(10, np.arange(-4, 1, dtype=float)),
                            'max_iter': np.arange(100, 450, 50, dtype = int ),  # integer valued parameter
                            'solver': ['sag', 'saga', 'newton-cg'],  # categorical parameter
                        },
                        cv = cv
                     )

lgr_gs.fit(Xtrain, ytrain)

print("params    : %s" % lgr_gs.best_params_)
print("val. score: %s" % lgr_gs.best_score_)
print("test score: %s" % lgr_gs.score(Xtest, ytest))

In [ ]:
svc_gs_df = get_hyperparameter_scores( svc_gs )
svc_rs_df = get_hyperparameter_scores( svc_rs )
svc_as_df = get_hyperparameter_scores( svc_as )

svc_scores = pd.concat( [ svc_gs_df.head( 1 ), svc_rs_df.head( 1 ), svc_as_df.head( 1 ) ] )

svc_scores.rename( index = { svc_scores.index[ 0 ]: 'SVC-GS', 
                             svc_scores.index[ 1 ]: 'SVC-RS',
                             svc_scores.index[ 2 ]: 'SVC-AS'
                           } 
                  ,inplace = True 
                 )

svc_scores.head()

In [ ]:
lgr_as_df  = get_hyperparameter_scores( lgr_as )
lgr_rs_df  = get_hyperparameter_scores( lgr_rs )
lgr_gs_df  = get_hyperparameter_scores( lgr_gs )

lgr_scores = pd.concat( [ lgr_gs_df.head( 1 ), lgr_rs_df.head( 1 ), lgr_as_df.head( 1 ) ] )

lgr_scores.rename( index = { lgr_scores.index[ 0 ]: 'LGR-GS', 
                             lgr_scores.index[ 1 ]: 'LGR-RS',
                             lgr_scores.index[ 2 ]: 'LGR-AS'
                           } 
                 , inplace = True )
lgr_scores.head()

In [ ]:
result_scores = pd.concat( [ svc_scores, lgr_scores ] )
result_scores.head( 10 )

In [ ]:
a             = 0.01
rope_interval = [-a, a]

n_train = len(list(cv.split(Xtrain, ytrain))[0][0])
n_test  = len(list(cv.split(Xtrain, ytrain))[0][1])

In [ ]:
from scipy.stats import t


def corrected_std(differences, n_train, n_test):
    """Corrects standard deviation using Nadeau and Bengio's approach.

    Parameters
    ----------
    differences : ndarray of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    corrected_std : float
        Variance-corrected standard deviation of the set of differences.
    """
    # kr = k times r, r times repeated k-fold crossvalidation,
    # kr equals the number of times the model was evaluated
    kr = len(differences)
    corrected_var = np.var(differences, ddof=1) * (1 / kr + n_test / n_train)
    corrected_std = np.sqrt(corrected_var)
    return corrected_std


def compute_corrected_ttest(differences, df, n_train, n_test):
    """Computes right-tailed paired t-test with corrected variance.

    Parameters
    ----------
    differences : array-like of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    df : int
        Degrees of freedom.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    t_stat : float
        Variance-corrected t-statistic.
    p_val : float
        Variance-corrected p-value.
    """
    mean = np.mean(differences)
    std = corrected_std(differences, n_train, n_test)
    t_stat = mean / std
    p_val = t.sf(np.abs(t_stat), df)  # right-tailed t-test
    return t_stat, p_val

In [ ]:
model_scores = result_scores.copy()
model_scores = model_scores[ [ 'mean_test_score', 'std_test_score' ] ]
model_scores.head( 10 )

In [ ]:
pairwise_bayesian = []

for model_i, model_k in combinations(range(len(model_scores)), 2):
    model_i_scores = model_scores.iloc[model_i].values
    model_k_scores = model_scores.iloc[model_k].values
    differences = model_i_scores - model_k_scores
    t_post = t(
        differences.shape[ 0 ] - 1, loc=np.mean(differences), scale=corrected_std(differences, n_train, n_test)
    )
    worse_prob = t_post.cdf(rope_interval[0])
    better_prob = 1 - t_post.cdf(rope_interval[1])
    rope_prob = t_post.cdf(rope_interval[1]) - t_post.cdf(rope_interval[0])

    pairwise_bayesian.append([model_scores.index[model_i], model_scores.index[model_k],
                              worse_prob, better_prob, rope_prob])

pairwise_bayesian_df = pd.DataFrame(
    pairwise_bayesian, columns=[ "model_1", "model_2", "worse_prob", "better_prob", "rope_prob"]
).round(3)

In [ ]:
pairwise_bayesian_df.sort_values( by = [ 'better_prob' ], ascending = False )

In [ ]:
lgr = LogisticRegression( C        = lgr_gs.best_params_[ 'C' ],
                          max_iter = lgr_gs.best_params_[ 'max_iter' ],
                          solver   = lgr_gs.best_params_[ 'solver' ],
                          tol      = lgr_gs.best_params_[ 'tol' ]
                        ) 
lgr.fit( Xtrain, ytrain )

y_pred = lgr.predict( Xtest )

cm = confusion_matrix( ytest, y_pred )
title = 'Confusion matrix for Logistic Regression'
disp = plot_confusion_matrix(lgr,
                             Xtest,
                             ytest,
                             display_labels = iris.target_names,
                             cmap           = plt.cm.Blues,
                             normalize      = None,
                            )
disp.ax_.set_title( title );

In [ ]:
pred_fn = lgr.decision_function
np.random.seed( 0 )
lgr_exp = KernelShap( pred_fn )
lgr_exp.fit( Xtrain )
lgr_shap_vals = lgr_exp.explain( Xtest, l1_reg = False )

shap.summary_plot( lgr_shap_vals.shap_values, Xtest, iris.feature_names, class_names = iris.target_names )

In [ ]:
svc = SVC( C      = svc_rs.best_params_[ 'C' ],
           degree = svc_rs.best_params_[ 'degree' ],
           gamma  = svc_rs.best_params_[ 'gamma' ],
           kernel = svc_rs.best_params_[ 'kernel' ]
         )

svc.fit( Xtrain, ytrain )

y_pred = lgr.predict( Xtest )

cm = confusion_matrix( ytest, y_pred )
title = 'Confusion matrix for SVM'
disp = plot_confusion_matrix(svc,
                             Xtest,
                             ytest,
                             display_labels = iris.target_names,
                             cmap           = plt.cm.Blues,
                             normalize      = None,
                            )
disp.ax_.set_title( title );

In [ ]:
pred_fn = svc.decision_function
np.random.seed( 0 )
svc_exp = KernelShap( pred_fn )
svc_exp.fit( Xtrain )
svc_shap_vals = svc_exp.explain( Xtest, l1_reg = False )

shap.summary_plot( svc_shap_vals.shap_values, Xtest, iris.feature_names, class_names = iris.target_names )